# Q6_Answer

对于上一题，考虑铝凝固过程中的潜热释放，计算温度分布随时间变化规律。提示：可采用温度回升法、等效热容法或求解焓方程的方法。

## 第一步：初始化相关参数

In [1]:
import numpy as np

node_size = 4  # 网格大小
col_len = 16 * node_size  # 列数
row_len = 16 * node_size  # 行数
node_num = col_len * row_len  # 网格数量
T_new = np.empty([row_len, col_len])  # 记录新温度
T_old = np.empty([row_len, col_len])  # 记录旧温度
H_new = np.empty([row_len, col_len])  # 记录新焓
H_old = np.empty([row_len, col_len])  # 记录旧焓

T_down = 25.0  # 下边界侧温度
T_old[:-node_size, :3*node_size] = 50
T_old[:-node_size, -3*node_size:] = 50
T_old[:-node_size, 3*node_size:-3*node_size] = 700
T_old[-node_size:, :] = T_down

max_loops = 3  # 迭代次数
step_size = 1  # 保存步长

In [2]:
thermal_conductivity = 237  # 导热系数 (W/(m·K)) 铝导热系数为 80
density = 2700  # 密度 (kg/m^3) 铝密度为 7800
specific_heat_capacity = 880  # 比热容 (J/kg*℃)
latent_heat = 3.9*pow(10,5) # 结晶潜热 (J/kg)

T_l = T_s = 660.4  # 固相液相线温度，纯铝两者一致(℃)

由$Q2(1.4)$ 得：$\Delta t = 0.5 \cdot \frac{\rho C_p \Delta x^2}{2\lambda}\tag{1.1}$

In [3]:
delta_x = 0.08/col_len  # 网格长度 (m)
print("delta_x: ", delta_x, "m")

delta_t = 0.001*(density*specific_heat_capacity*pow(delta_x, 2))\
    / (2*thermal_conductivity)
print("delta_t: ", delta_t, "s")

delta_x:  0.00125 m
delta_t:  7.83227848101266e-06 s


## 第二步：定义有限差分算法（求解焓方程）（矩阵法）

由：$$\frac{H^{t+\Delta t}_i - H^t_i}{\Delta t}=\frac{\lambda(T^t_{i-1}-2T^t_i+T^t_{i+1})}{\rho(\Delta x)^2}\tag{2.1}$$

即：$$H^{t+\Delta t}_i=\frac{\lambda\Delta t(T^t_{i-1}-2T^t_i+T^t_{i+1})}{\rho(\Delta x)^2}+ H^t_i\tag{2.2}$$

为了节省在 `for` 中的计算量

我们设置一个缓存变量：$$h\_cache = \frac{\lambda\Delta t}{\rho(\Delta x)^2}\tag{2.3}$$ 

则上式化为：$$H^{t+\Delta t}_i=h\_cache \cdot (T^t_{i-1}-2T^t_i+T^t_{i+1})+ H^t_i\tag{2.4}$$

In [4]:
h_cache = thermal_conductivity*delta_t/(density*pow(delta_x,2))
print("h_cache: ", h_cache)

h_cache:  0.44000000000000006


In [5]:
from numba import jit  # 使用 numba 库提高计算性能


@jit(nopython=True)
def H_solve(H_new, H_old,T_old):
    # 先计算去边矩阵
    H_new[1:-1, 1:-1] = h_cache * (T_old[:-2, 1:-1] + T_old[2:, 1:-1] +
                                    T_old[1:-1, :-2] + T_old[1:-1, 2:] -
                                    4 * T_old[1:-1, 1:-1]) + H_old[1:-1, 1:-1]

    H_new[:, 0] = H_new[:, 1]  # 左边界
    H_new[:, -1] = H_new[:, -2]  # 右边界
    H_new[-1, :] = H_new[-2, :]  # 下边界
    H_new[0, :] = H_new[1, :]  # 下边界
    
    H_new[:] = 1
    return H_new

由：$$H=C_pT+f_lL_h\tag{2.5}$$

可得：$$T_{new}=\frac{H_{new}-f_lL_h}{C_p}\tag{2.6}$$

又：$$f_l=1-f_s\tag{2.7}$$

在二元体系中：$$f_s = \frac{T_l-T}{(1-k_0)(T_m-T)}\tag{2.8}$$

而本题涉及到的材料为纯铝，

因此：$$f_s = \frac{H_l-H}{H_l-H_s}\tag{2.9}$$

由 $(2.7)$：

$$f_l=\frac{(H_l-H_s) - (H_l-H)}{H_l-H_s}=\frac{H-H_s}{H_l-H_s}\tag{2.10}$$

因此我们要求出 $H_s, H_l$，而 $H_s$ 对应 $f_l = 0$，$H_l$ 对应 $f_l = 1$

即：$$H_s=C_pT_s\tag{2.11}$$ 

$$H_l=C_pT_l+L_h\tag{2.12}$$

同时令：$\Delta H = H_l - H_s\tag{2.13}$

In [6]:
H_s = specific_heat_capacity*T_s
H_l = specific_heat_capacity*T_l+latent_heat
print("H_s: ", H_s)
print("H_l: ", H_l)
delta_H = H_l - H_s
print("delta_H: ", delta_H)

H_s:  581152.0
H_l:  971152.0
delta_H:  390000.0


In [7]:
@jit(nopython=True)
def H_solve_init(H_new,T_old):
    # 先计算去边矩阵
    H_new[:-node_size, 3*node_size:-3*node_size] = thermal_conductivity*T_old[:-node_size, 3*node_size:-3*node_size]+latent_heat
    H_new[:-node_size, :3*node_size] = thermal_conductivity*T_old[:-node_size, :3*node_size]
    H_new[:-node_size, -3*node_size:] = thermal_conductivity*T_old[:-node_size, -3*node_size:]
    H_new[-node_size:, :] = thermal_conductivity*T_old[-node_size:, :]
    
    return H_new

联立 $(2.13)(2.10)(2.6)$ 可得：

$$T_{new}=\frac{H_{new}-\frac{H_{new} -H_s}{\Delta H}\cdot L_h}{C_p}\tag{2.14}$$

In [8]:
@jit(nopython=True)
def thermal_conductivity_method(T_new, T_old, H_new, H_old):
    # de_edge_matrix_size = (slice(1,row_len-node_size),slice(1,col_len-1))
    H_old = H_solve(H_new, H_old,T_old).copy()
    
    # print(H_old)
    H_old[:] = 9711520.0
    T_new = ((H_old-((H_old-H_s)/delta_H)*latent_heat)/specific_heat_capacity).copy()
    T_new[-node_size:, :] = T_down
    
    return T_new,H_old

## 第三步：启动循环

In [9]:
thermal_data = []
append = thermal_data.append
H_old = H_solve_init(H_new,T_old).copy()
H_old[:] = 1
append(T_old)
for loops in range(max_loops+1):
    T_old,H_old = thermal_conductivity_method(T_new, T_old, H_new, H_old)
    if loops % step_size == 0:
        append(T_old)  # 由于 loop 小于 1w，移去判断

## 第五步：绘图

In [10]:
# 导入绘图库
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [11]:
data = [go.Heatmap(visible=False, zmin=25, zmax=700, z=t,
                   hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> ") for t in thermal_data]

data1 = [go.Surface(visible=False, z=t,
                    hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> ") for t in thermal_data]

data1[0]['visible'] = data[0]['visible'] = True

In [12]:
steps = []
data_len = len(data)
for i in range(data_len):
    step = dict(
        method='restyle',
        args=['visible', [False] * data_len],
        label=i*step_size
    )
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "loop: "},
    steps=steps
)]

layout = dict(sliders=sliders, width=800, height=800,
              yaxis=dict(autorange='reversed'), xaxis=dict(side='top'))
fig = dict(data=data, layout=layout)
fig1 = dict(data=data1, layout=layout)
plotly.offline.iplot(fig)
# plotly.offline.iplot(fig1)